In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import numpy as np
import torch
import torch.nn as nn

from pideq.net import PINN, PIDEQ
from pideq.utils import load_from_wandb
from pideq.trainer import PINNTrainer

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [17]:
pinn = load_from_wandb(PINN(np.pi / 2, n_hidden=4, n_nodes=100), '2n1n4ml0', project='pideq-nls', model_fname='model_last')
pinn.eval().to(device)

deq = PIDEQ.from_pinn(pinn)
deq.eval().to(device)

PIDEQ(
  (B): Linear(in_features=400, out_features=400, bias=True)
  (A): Linear(in_features=2, out_features=400, bias=True)
  (h): Linear(in_features=400, out_features=2, bias=True)
)

In [34]:
tr = PINNTrainer(PINN(np.pi / 2))
tr.prepare_data()
(t, x), Y = tr.val_data

h = torch.sqrt(Y[:,0]**2 + Y[:,1]**2)

In [35]:
with torch.no_grad():
    Y_pred_pinn = pinn(t, x)
    Y_pred_deq = deq(t, x)

    h_pred_pinn = torch.sqrt(Y_pred_pinn[:,0]**2 + Y_pred_pinn[:,1]**2)
    h_pred_deq = torch.sqrt(Y_pred_deq[:,0]**2 + Y_pred_deq[:,1]**2)

In [39]:
torch.linalg.norm(h - h_pred_pinn, 2) / torch.linalg.norm(h_pred_pinn)

tensor(0.0173, device='cuda:0')

In [38]:
torch.linalg.norm(h - h_pred_deq, 2) / torch.linalg.norm(h_pred_deq)

tensor(0.0173, device='cuda:0')